In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import datetime
import calendar
from dateutil.relativedelta import relativedelta

class PrcRcdMigracionTipoCreditoSBS():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      #fecha_actualizacion = datetime.now()
      #self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def MigracionTipoCreditoSBS(self):
    
    fechaProcesoRcc = self.fecha_proceso.replace(day=1) - timedelta(days=1)
    fechaProcesoRcc_str = fechaProcesoRcc.strftime('%Y-%m-%d') 
    fechaProcesoIndicadorFin = fechaProcesoRcc.strftime('%Y-%m-%d')
    
    #Reproceso
    eliminaIndicador = "DELETE CLIENTE_DEUDOR_DETALLE WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "' AND INDICADOR IN ('ET','TC')"
    eliminaIndicadorDetalle = "DELETE CLIENTE_DEUDOR_PRODUCTO_DETALLE WHERE FECHA_PROCESO = '" + self.fecha_proceso_str + "' AND INDICADOR = 'TC'"
    self.cursor.execute(eliminaIndicador)
    self.cursor.execute(eliminaIndicadorDetalle)
    self.connection.commit()
    
    #Proceso
    #obtenemos la configuracion de los indicadores endeudamiento y tipo de credito sbs
    indicador = """SELECT B.INDICADOR,
            B.TIPO_CREDITO_SBS,
            C.CUENTA_CONTABLE
    FROM INDICADOR_DETALLE B
    JOIN RCC_CUENTA_CONTABLE C ON C.CONCEPTO_SALDO = B.CONCEPTO_SALDO
    AND B.SITUACION_CREDITO = C.SITUACION_CREDITO
    WHERE INDICADOR IN ('ET', 'TC')
    """   
    sql_indicador = pd.read_sql_query(indicador, self.connection)
    df_indicador = pd.DataFrame(sql_indicador)
    df_indicador['CUENTA_CONTABLE_LEN'] = df_indicador['CUENTA_CONTABLE'].str.len()
    
    meses = 1
    #obtenemos los saldos de los ultimos 6 meses del cliente en el SF
    while meses < 6:
      rccSaldos1 = """SELECT A.ID CLIENTE_DEUDOR, B.ID RCC, C.ID RCC_SALDOS, C.EMPRESA, C.TIPO_CREDITO_SBS, C.CUENTA_CONTABLE, C.DIAS_ATRASO, C.IMPORTE, C.CLASIFICACION_DEUDOR
    FROM CLIENTE_DEUDOR A JOIN RCC B
    ON B.FECHA_PROCESO = """"'"+ fechaProcesoRcc_str + """'
    AND B.CODIGO_DEUDOR_SBS = A.CODIGO_DEUDOR_SBS
    JOIN RCC_SALDOS C ON B.ID = C.RCC"""
      sql_rccSaldos1 = pd.read_sql_query(rccSaldos1, self.connection)
      df_rccSaldos1 = pd.DataFrame(sql_rccSaldos1)
      
      rccSaldos2 = """SELECT A.ID CLIENTE_DEUDOR, B.ID RCC, C.ID RCC_SALDOS, C.EMPRESA, C.TIPO_CREDITO_SBS, C.CUENTA_CONTABLE, C.DIAS_ATRASO, C.IMPORTE, C.CLASIFICACION_DEUDOR
    FROM CLIENTE_DEUDOR A JOIN RCC B
    ON B.FECHA_PROCESO = """"'"+ fechaProcesoRcc_str + """'
    AND B.TIPO_DOCUMENTO_IDENTIDAD = A.TIPO_DOCUMENTO_IDENTIDAD
    AND B.DOCUMENTO_IDENTIDAD = A.DOCUMENTO_IDENTIDAD
    AND B.TIPO_PERSONA = A.TIPO_PERSONA
    JOIN RCC_SALDOS C ON B.ID = C.RCC"""
      sql_rccSaldos2 = pd.read_sql_query(rccSaldos2, self.connection)
      df_rccSaldos2 = pd.DataFrame(sql_rccSaldos2)
      
      rccSaldos3 = """SELECT A.ID CLIENTE_DEUDOR, B.ID RCC, C.ID RCC_SALDOS, C.EMPRESA, C.TIPO_CREDITO_SBS, C.CUENTA_CONTABLE, C.DIAS_ATRASO, C.IMPORTE, C.CLASIFICACION_DEUDOR
    FROM CLIENTE_DEUDOR A JOIN RCC B
    ON B.FECHA_PROCESO = """"'"+ fechaProcesoRcc_str + """'
    AND B.TIPO_DOCUMENTO_TRIBUTARIO = A.TIPO_DOCUMENTO_TRIBUTARIO
    AND B.DOCUMENTO_TRIBUTARIO = A.DOCUMENTO_TRIBUTARIO
    JOIN RCC_SALDOS C ON B.ID = C.RCC"""
      sql_rccSaldos3 = pd.read_sql_query(rccSaldos3, self.connection)
      df_rccSaldos3 = pd.DataFrame(sql_rccSaldos3)
      
      #unimos los dataframes y quitamos los duplicados
      df_rccSaldos = pd.concat([df_rccSaldos1,df_rccSaldos2, df_rccSaldos3]).drop_duplicates()
      
      #unimos los dataframes
      saldosCuentaContable = pd.merge(df_indicador,df_rccSaldos, on = ['TIPO_CREDITO_SBS','CUENTA_CONTABLE'], how='inner')
      
      #substraemos la cuenta contable con los caracteres necesarios
      saldosCuentaContable['CUENTA_CONTABLE'] = saldosCuentaContable['CUENTA_CONTABLE'].str[:'CUENTA_CONTABLE_LEN'] #OBS 
      
      #unimos por cuenta contable
      cliente_ET = pd.merge(df_indicador,saldosCuentaContable, on = 'CUENTA_CONTABLE', how='inner') #OBS
      #print(cliente_ET)
      
      #sumamos el importe por cliente e indicador
      #cliente_ET = cliente_ET.groupby(['CLIENTE_DEUDOR','INDICADOR_x']).agg('sum')
      #df_cliente_ET = pd.DataFrame(cliente_ET)
      
      #agregamos campos en duro
      cliente_ET['FECHA_INDICADOR'] = fechaProcesoRcc
      cliente_ET['VALOR'] = ''
      cliente_ET['FECHA_PROCESO'] = self.fecha_proceso_str
      
      df_cliente_ET = cliente_ET[['CLIENTE_DEUDOR','INDICADOR_x','FECHA_INDICADOR','IMPORTE','FECHA_PROCESO']]
      values =  cliente_ET.values.tolist()
      if values:
        #Insertando datos
        insert = "INSERT INTO CLIENTE_DEUDOR_DETALLE() VALUES (?,?,?,?,?,?)"
        self.cursor.fast_executemany = True
        self.cursor.executemany(insert, cliente_ET.values.tolist())
        self.connection.commit()   
      meses = meses + 1
      fechaProcesoRcc = fechaProcesoRcc.replace(day=1) - timedelta(days=1)
      fechaProcesoRcc_str = fechaProcesoRcc.strftime('%Y-%m-%d')
    self.cursor.close()       
    
#objeto = PrcRcdMigracionTipoCreditoSBS(4)
#objeto.MigracionTipoCreditoSBS()